In [20]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder \
    .appName("Footbal project") \
    .getOrCreate()

results_df = spark.read.option("header", "true").csv("results.csv")
goals_df = spark.read.option("header", "true").csv("goalscorers.csv")

results_df.show(5)
goals_df.show(5)


+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  FALSE|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  FALSE|
|1874-03-07| Scotland|  England|         2|         1|  Friendly|Glasgow|Scotland|  FALSE|
|1875-03-06|  England| Scotland|         2|         2|  Friendly| London| England|  FALSE|
|1876-03-04| Scotland|  England|         3|         0|  Friendly|Glasgow|Scotland|  FALSE|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
only showing top 5 rows
+----------+---------+---------+---------+----------------+------+--------+-------+
|      date|home_team|away_team|     team|          scorer|minute|own_goa

Prvi zadatak: najbolja i najgora gol razlika Srbije 

In [ ]:
rdd = results_df.rdd.map(lambda row: (
    row['home_team'], 
    row['away_team'], 
    int(row['home_score']), 
    int(row['away_score'])
))

serbia_rdd = rdd.filter(lambda x: x[0] == 'Serbia' or x[1] == 'Serbia')

def map_goal_diff(match):
    home, away, home_score, away_score = match
    if home == 'Serbia':
        opponent = away
        diff = home_score - away_score
    else:
        opponent = home
        diff = away_score - home_score
    return (opponent, diff)

mapped = serbia_rdd.map(map_goal_diff)
reduced = mapped.reduceByKey(lambda a, b: a + b)

best = reduced.max(key=lambda x: x[1])
worst = reduced.min(key=lambda x: x[1])

print("Najbolja gol razlika sa Srbijom:", best)
print("Najgora gol razlika sa Srbijom:", worst)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 60.0 failed 1 times, most recent failure: Lost task 0.0 in stage 60.0 (TID 47) (Nikola executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed). Consider setting 'spark.sql.execution.pyspark.udf.faulthandler.enabled' or'spark.python.worker.faulthandler.enabled' configuration to 'true' for the better Python traceback.
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:621)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:35)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:945)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(Pytho